In [22]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

In [23]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [24]:
# Get your API keys from openai, you will need to create an account.
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = ""

In [33]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [34]:
# location of the pdf file/files.
reader = PdfReader('/content/gdrive/My Drive/mini.pdf')

In [35]:
reader

In [36]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [37]:
# raw_text

In [38]:
raw_text[:100]

'e\xadCourt,\xa0Pathanamthitta\xa0District,Kerala. Page\xa01\xa0of\xa015DISTRICT COURT, PATHANAMTHITTA\nKnow the IPC (In'

In [39]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits.

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [40]:
len(texts)

50

In [41]:
texts[0]

'e\xadCourt,\xa0Pathanamthitta\xa0District,Kerala. Page\xa01\xa0of\xa015DISTRICT COURT, PATHANAMTHITTA\nKnow the IPC (Indian Penal Code) Sections.\n        Section1.(Introduction)TitleandextantofoperationoftheCode\nSection 2.(Introduction) Punishment of offences committed within India\nSection 3.(Introduction) Punishment of offences committed beyond, but which by law may be tried within, India\nSection 4.(Introduction) Extension of Code to extra-territorial offences\nSection 5.(Introduction) Certain laws not to be affected by this Act\nSection 6.(General explanations) Definitions in the Code to be understood subject to exceptions\nSection 7.(General explanations) Sense of expression once explained\nSection 8.(General explanations) Gender\nSection 9.(General explanations) Number\nSection 10.(General explanations) Man, Woman\nSection 11.(General explanations) Person\nSection 12.(General explanations) Public\nSection 13.(General explanations) Queen\nSection 14. (General explanations)Serva

In [42]:
texts[1]

'Section 11.(General explanations) Person\nSection 12.(General explanations) Public\nSection 13.(General explanations) Queen\nSection 14. (General explanations)Servant of Government\nSection 15. (General explanations)British India\nSection 16. (General explanations)Government of India\nSection 17. (General explanations)Government\nSection 18.(General explanations) India\nSection 19.(General explanations) Judge\nSection 20.(General explanations) Court of Justice\nSection 21. (General explanations)Public Servant\nSection 22.(General explanations)Moveable property\nSection 23.(General explanations) Wrongful gain\nSection 24.(General explanations) Dishonestly\nSection 25. (General explanations)Fraudulently\nSection 26.(General explanations) Reason to believe\nSection 27.(General explanations) Property in possession of wife, clerk or servant\nSection 28.(General explanations)Counterfeit\nSection 29.(General explanations)Document\nSection 29A.(General explanations)Electronic record'

In [43]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [44]:
docsearch = FAISS.from_texts(texts, embeddings)

In [45]:
docsearch

In [46]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [47]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [48]:
query = "which IPC law comes when i get hit by a car"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' It depends on the circumstances of the case. If the driver of the car intentionally hit you, it could fall under Section 320 (grievous hurt) or Section 333 (voluntarily causing grievous hurt to deter public servant from his duty). If it was a reckless or negligent act, it could fall under Section 337 (causing hurt by act endangering life or personal safety of others). However, it is ultimately up to the judge and the evidence presented in court to determine the specific IPC law that applies in this situation.'

In [50]:
query = "which IPC law comes when i get hit by a car and the person was drinking and driving in the car. explain in details"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Section 304A of the Indian Penal Code relates to causing death by negligence. In the given scenario, if a person is hit by a car and the driver was under the influence of alcohol, it can be considered as a case of causing death by negligence. This section states that whoever causes the death of a person by doing any rash or negligent act not amounting to culpable homicide, shall be punished with imprisonment of either description for a term which may extend to two years or with fine or with both. However, if the act is done with the knowledge that it is likely to cause death or with the intention of causing death, then it may be considered as culpable homicide not amounting to murder under Section 299 of the IPC. In such cases, the punishment may extend to imprisonment for life or for a term which may extend to 10 years, along with a fine. Additionally, the driver may also face charges under Section 185 of the Motor Vehicles Act for driving under the influence of alcohol. This sectio